In [1]:
# import libraries
import os, sys
import string
import pandas as pd
try:
    import nltk
except:
    !conda install --yes --prefix {sys.prefix} nltk
    import nltk

# import modules
from utils.preprocessing import get_dataframe, label_encoding

In [2]:
# ver conteúdos da pasta
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'BOA_project_description.pdf',
 'corpora',
 'corpora_preprocessing.ipynb',
 'LICENSE',
 'README.md',
 'requirements.txt',
 'utils']

In [3]:
authors = os.listdir("corpora/train")
authors

['AlmadaNegreiros',
 'CamiloCasteloBranco',
 'EcaDeQueiros',
 'JoseRodriguesSantos',
 'JoseSaramago',
 'LuisaMarquesSilva']

In [4]:
for author in authors:
    for excerto in [x for x in os.listdir(f"corpora/train/{author}") if x.endswith(".txt")]:
        print(f"corpora/train/{author}/{excerto}")

corpora/train/AlmadaNegreiros/pg22615.txt
corpora/train/AlmadaNegreiros/pg22730.txt
corpora/train/AlmadaNegreiros/pg22801.txt
corpora/train/AlmadaNegreiros/pg22802.txt
corpora/train/AlmadaNegreiros/pg22969.txt
corpora/train/AlmadaNegreiros/pg23133.txt
corpora/train/AlmadaNegreiros/pg23620.txt
corpora/train/AlmadaNegreiros/pg23879.txt
corpora/train/AlmadaNegreiros/pg23961.txt
corpora/train/CamiloCasteloBranco/24691-0.txt
corpora/train/CamiloCasteloBranco/34756-0.txt
corpora/train/CamiloCasteloBranco/pg16425.txt
corpora/train/CamiloCasteloBranco/pg17927.txt
corpora/train/CamiloCasteloBranco/pg19375.txt
corpora/train/CamiloCasteloBranco/pg21406.txt
corpora/train/CamiloCasteloBranco/pg23203.txt
corpora/train/CamiloCasteloBranco/pg23345.txt
corpora/train/CamiloCasteloBranco/pg23346.txt
corpora/train/CamiloCasteloBranco/pg24339.txt
corpora/train/CamiloCasteloBranco/pg25844.txt
corpora/train/CamiloCasteloBranco/pg26017.txt
corpora/train/CamiloCasteloBranco/pg26103.txt
corpora/train/CamiloCast

In [5]:
train = get_dataframe(path_to_train="corpora/train/", author_list=authors,
                      preserve_blank_lines=False, join_every_line=True,
                      separator="")
train

,text,author
0,Title: A Scena do OdioAuthor: José de Almada N...,AlmadaNegreiros
1,Title: O Jardim da PierretteAuthor: José de Al...,AlmadaNegreiros
2,Title: A Invenção do Dia ClaroAuthor: José de ...,AlmadaNegreiros
3,Title: LitoralA Amadeo de Souza CardozoAuthor:...,AlmadaNegreiros
4,EXPOSIÇÃO+amadeode souzacardoso+LIGA NAVAL DE ...,AlmadaNegreiros
...,...,...
58,Títuloe-medoAutora (inspiradíssima)Luísa Marqu...,LuisaMarquesSilva
59,"LISBOA 2050Lisboa, ano de 2050. Um Agosto tão ...",LuisaMarquesSilva
60,TítuloUm passeio pelo infernoAutoraLuísa Marqu...,LuisaMarquesSilva
61,TítuloRapsódia sem dó (maior)AutoraLuísa Marqu...,LuisaMarquesSilva


In [6]:
# codificação da coluna y para fornecer ao futuro modelo
train = label_encoding(train, "author")
train

,text,author
0,Title: A Scena do OdioAuthor: José de Almada N...,0
1,Title: O Jardim da PierretteAuthor: José de Al...,0
2,Title: A Invenção do Dia ClaroAuthor: José de ...,0
3,Title: LitoralA Amadeo de Souza CardozoAuthor:...,0
4,EXPOSIÇÃO+amadeode souzacardoso+LIGA NAVAL DE ...,0
...,...,...
58,Títuloe-medoAutora (inspiradíssima)Luísa Marqu...,5
59,"LISBOA 2050Lisboa, ano de 2050. Um Agosto tão ...",5
60,TítuloUm passeio pelo infernoAutoraLuísa Marqu...,5
61,TítuloRapsódia sem dó (maior)AutoraLuísa Marqu...,5


In [7]:
def remove_punctuation(text):
    """
        Greedy removal of all the punctuation from a list of text lines.
        However, it also removes the email and website punctuation, making
        them hard to recognise.
        Returns a list of text lines without punctuation.
    """
    no_punct_text = []
    for line in text:
        no_punct = "".join([char for char in line if char not in string.punctuation])
        no_punct_text.append(no_punct)
    return "".join(no_punct_text)


# cada vez que uma nova função for criada, introduzi-la em "clean_text()"
def clean_text(df, text_col=str):
    """
        Compiles all the preprocessing functions inside a single function.
    """
    df[text_col] = df[text_col].apply(remove_punctuation)
    return df

In [8]:
train = clean_text(train, "text")
train

,text,author
0,Title A Scena do OdioAuthor José de Almada Neg...,0
1,Title O Jardim da PierretteAuthor José de Alma...,0
2,Title A Invenção do Dia ClaroAuthor José de Al...,0
3,Title LitoralA Amadeo de Souza CardozoAuthor J...,0
4,EXPOSIÇÃOamadeode souzacardosoLIGA NAVAL DE LI...,0
...,...,...
58,TítuloemedoAutora inspiradíssimaLuísa Marques ...,5
59,LISBOA 2050Lisboa ano de 2050 Um Agosto tão ge...,5
60,TítuloUm passeio pelo infernoAutoraLuísa Marqu...,5
61,TítuloRapsódia sem dó maiorAutoraLuísa Marques...,5


[in progress] ***brainstorm* de features interessantes:**
- número de caracteres por linha
- número de caracteres especiais por linha
- número de vírgulas por frase
- número de reticências por linha
